các bước của tiễn xử lí sẽ là 
- đầu tiên là làm sạch và biến đổi dữ liệu
	- tích hợp dữ liệu cho mỗi nguồn
	- làm sạch dữ liệu từ mỗi nguồn dữ liệu	
	- transform dữ liệu thành một dạng quy chuẩn nhất định
- tích hợp dữ liệu thì nhiều nguồn

# facebook

In [2]:
import pandas as pd 
import re
import os
import json

In [2]:
# Đường dẫn tới thư mục gốc
base_dir = r"C:\working\job_rcm\data\facebook\2024"

In [3]:
# Danh sách để lưu tất cả các DataFrames
all_posts = []

In [ ]:
# Duyệt qua tất cả các thư mục con và file trong base_dir
for root, dirs, files in os.walk(base_dir):
    for file in files:
        # Kiểm tra xem file có phải là file JSON không
        if file == 'data.json':
            file_path = os.path.join(root, file)
            # Đọc dữ liệu từ file JSON
            with open(file_path, 'r', encoding='utf-8') as f:
                data = json.load(f)
                all_posts.append(data)  # Thêm dữ liệu của file JSON vào danh sách

# Lưu toàn bộ dữ liệu vào một file JSON lớn
output_path = r'C:\working\job_rcm\data\preprocessed\facebook\merged_data.json'
with open(output_path, 'w', encoding='utf-8') as f:
    json.dump(all_posts, f, ensure_ascii=False, indent=4)

print(f"Dữ liệu đã được hợp nhất thành công và lưu vào file '{output_path}'")

In [4]:
import pandas as pd

# Đường dẫn tới file JSON đã hợp nhất
json_path = r"C:\working\job_rcm\data\preprocessed\facebook\merged_data.json"

# Đọc dữ liệu từ file JSON
df = pd.read_json(json_path)

In [5]:
df['post_date'].value_counts()

post_date
+1                                     1152
Thứ Ba, 18 Tháng 6, 2024 lúc 11:51        8
Thứ Tư, 11 Tháng 5, 2022 lúc 19:29        6
Thứ Năm, 3 Tháng 10, 2024 lúc 08:25       6
Thứ Hai, 23 Tháng 9, 2024 lúc 10:54       6
                                       ... 
Thứ Hai, 16 Tháng 9, 2024 lúc 11:28       1
Thứ Năm, 12 Tháng 9, 2024 lúc 10:41       1
Thứ Sáu, 30 Tháng 8, 2024 lúc 12:00       1
Thứ Sáu, 1 Tháng 7, 2022 lúc 14:38        1
Chủ Nhật, 4 Tháng 6, 2023 lúc 11:42       1
Name: count, Length: 219, dtype: int64

In [ ]:

df['post_date'] = df['post_date'].apply(
    lambda x: re.sub(r"Thứ \w+, (\d+) Tháng (\d+), (\d+) lúc (\d+:\d+)", r"\3-\2-\1 \4", x) if pd.notnull(x) else x
)
df['post_date'] = pd.to_datetime(df['post_date'], format='%Y-%m-%d %H:%M', errors='coerce')



In [ ]:
# # Điền giá trị NaT bằng giá trị phía trên trừ đi 30 phút trong mỗi group
# df['post_date'] = df.groupby('group_id')['post_date'].transform(lambda x: x.ffill() - pd.Timedelta(minutes=30))

# cách làm này sẽ bị lặp rất nhiều 

In [ ]:

def fill_nat_with_timedelta(group):
    last_valid = None
    result = []
    for date in group:
        if pd.isna(date):
            last_valid -= pd.Timedelta(minutes=30)
            result.append(last_valid)
        else:
            last_valid = date
            result.append(last_valid)
    return result

df['post_date'] = df.groupby('group_id')['post_date'].transform(fill_nat_with_timedelta)


In [40]:
df

,post_href,author_name,hr_id,group_id,scrape_date,post_date,content,images
0,https://www.facebook.com/groups/16032510932029...,Phương Phạm,https://www.facebook.com/groups/160325109320298/,https://www.facebook.com/groups/160325109320298/,2024-10-24 20:10:29,2024-10-23 14:28:00,Inbox mình gửi Jd,[]
1,https://www.facebook.com/groups/16032510932029...,Lê Quỳnh,https://www.facebook.com/profile.php?id=100011...,https://www.facebook.com/groups/160325109320298/,2024-10-24 20:10:51,2024-10-24 09:43:00,TUYỂN DỤNG NHÂN VIÊN XỬ LÝ DỮ LIỆU HÌNH ẢNH - ...,[https://scontent-xsp1-1.xx.fbcdn.net/v/t39.30...
2,https://www.facebook.com/groups/16032510932029...,Nguyễn Võ Phương Trinh,https://www.facebook.com/groups/160325109320298/,https://www.facebook.com/groups/160325109320298/,2024-10-24 20:11:06,2024-10-23 18:00:00,[DAYONE ASIA (SUNNY DAYS PIANO)] - TUYỂN DỤNG\...,[https://scontent-xsp1-1.xx.fbcdn.net/v/t39.30...
3,https://www.facebook.com/groups/16032510932029...,Kim Lê,https://www.facebook.com/groups/160325109320298/,https://www.facebook.com/groups/160325109320298/,2024-10-24 20:11:21,2024-10-21 18:05:00,"Đừng săn Hồng Hài Nhi nữaaa, săn job XGame đi ...",[https://scontent-xsp1-2.xx.fbcdn.net/v/t39.30...
4,https://www.facebook.com/groups/16032510932029...,Phương Phạm,https://www.facebook.com/groups/160325109320298/,https://www.facebook.com/groups/160325109320298/,2024-10-24 20:16:53,2024-10-23 14:28:00,Inbox mình gửi Jd,[]
...,...,...,...,...,...,...,...,...
1811,https://www.facebook.com/groups/16032510932029...,Giang Nam,https://www.facebook.com/profile.php?id=100027...,https://www.facebook.com/groups/3272112773116278/,2024-10-26 09:00:20,2022-10-27 20:44:00,(HN)#DE 2.5+ năm exp. Có exp ELT. Pv online 1 ...,[]
1812,https://www.facebook.com/groups/16032510932029...,Thùy Emm,https://www.facebook.com/profile.php?id=100010...,https://www.facebook.com/groups/3272112773116278/,2024-10-26 09:00:39,2022-10-27 20:14:00,[Q7- HCM] - CƠ HỘI VIỆC LÀM CHO FRESHERS VỊ TR...,[https://scontent-xsp1-2.xx.fbcdn.net/v/t39.30...
1813,https://www.facebook.com/groups/16032510932029...,Vy Thúy,https://www.facebook.com/profile.php?id=100070...,https://www.facebook.com/groups/3272112773116278/,2024-10-26 09:00:58,2022-10-27 19:44:00,[HN]\nVIETINBANK đang chiêu mộ CHUYÊN GIA KIẾN...,[https://scontent-xsp1-1.xx.fbcdn.net/v/t39.30...
1814,https://www.facebook.com/groups/16032510932029...,Lê Phương Hồng,https://www.facebook.com/profile.php?id=100091...,https://www.facebook.com/groups/3272112773116278/,2024-10-26 09:01:18,2022-10-27 19:14:00,[HCM/Q1]\nJOB XỊN - DEAL XỊN đến từ AVENTRA GR...,[https://scontent-xsp1-1.xx.fbcdn.net/v/t39.30...


In [31]:
sample_structure = df.head(1).to_dict(orient='records')
sample_structure

[{'post_href': 'https://www.facebook.com/groups/160325109320298/user/100052727601101/?__cft__[0]=AZUKtCEADwRUBXdVIj-_XRyQ0o39MUJ-zWJ_Hzf0Zwd0WOk_f48ptUetE-6b_arBODzVwe_lF2aae_j409IlyQlRUXqDtlpLyqmonVXRuECqNLq9fClWNicYDECjgAG2hydzb_GtU1jNFO_5IQU64kILwaQy6AKrvtzt8GqdcZpjKWxTOKeywqwONSnV4GfPvMs&__tn__=%3C%2CP-R',
  'author_name': 'Phương Phạm',
  'hr_id': 'https://www.facebook.com/groups/160325109320298/',
  'group_id': 'https://www.facebook.com/groups/160325109320298/',
  'scrape_date': '2024-10-24 20:10:29',
  'post_date': Timestamp('2024-10-23 14:28:00'),
  'content': 'Inbox mình gửi Jd',
  'images': []}]

# linkedin

In [6]:
import pandas as pd 
import os 
import json
from datetime import datetime

In [40]:
all_ld_posts = []
base_dir = r'C:\working\job_rcm\data\linkedin'

In [ ]:
# Duyệt qua tất cả các thư mục con
for root, dirs, files in os.walk(base_dir):
    if 'Detailview' in root:  # Chỉ lấy các thư mục chứa "Detailview"
        for file in files:
            if file == 'data.json':
                file_path = os.path.join(root, file)
                with open(file_path, 'r', encoding='utf-8') as f:
                    data = json.load(f)
                    all_ld_posts.append(data)

# Lưu kết quả
output_path = r'C:\working\job_rcm\data\preprocessed\linkedin\merged_data.json'
with open(output_path, 'w', encoding='utf-8') as f:
    json.dump(all_ld_posts, f, ensure_ascii=False, indent=4)

print(f"Dữ liệu đã được hợp nhất thành công và lưu vào file '{output_path}'")

Dữ liệu đã được hợp nhất thành công và lưu vào file 'C:\working\job_rcm\data\preprocessed\linkedin\merged_data.json'


In [38]:
import json
import pandas as pd

linkedin_json_path = r'C:\working\job_rcm\data\preprocessed\linkedin\merged_data.json'

with open(linkedin_json_path, 'r', encoding='utf-8') as f:
    data = json.load(f)

all_ld_jobs = [item["job"] for item in data if "job" in item]

ld_df = pd.DataFrame(all_ld_jobs)
print(ld_df.head(1))

  company_name company_href           job_name                     region  \
0         None         None  Presales Engineer  Ho Chi Minh City, Vietnam   

  job_status                 scrape_date  \
0  available  2025-03-27T16:16:02.704842   

                                              hr_id  \
0  https://www.linkedin.com/in/luan-nguyen-62682847   

                                                  jd  
0  About the job\nMô tả công việc:\nNắm vững các ...  


In [39]:
ld_df['scrape_date'] = pd.to_datetime(ld_df['scrape_date'], format="%Y-%m-%dT%H:%M:%S.%f")
ld_df['scrape_date'] = ld_df['scrape_date'].dt.strftime("%Y-%m-%d %H:%M")
ld_df.head()

,company_name,company_href,job_name,region,job_status,scrape_date,hr_id,jd
0,None,None,Presales Engineer,"Ho Chi Minh City, Vietnam",available,2025-03-27 16:16,https://www.linkedin.com/in/luan-nguyen-62682847,About the job\nMô tả công việc:\nNắm vững các ...
1,GETS - Global Energy Technical services Compan...,https://www.linkedin.com/company/gets-global-e...,Sales Engineer,"District 9, Ho Chi Minh City, Vietnam",available,2025-03-27 16:16,https://www.linkedin.com/in/tinh-to-099921133,About the job\nMÔ TẢ CÔNG VIỆC / JOB DESCRIPTI...
2,LTD Material LLC,https://www.linkedin.com/company/ltd-material/...,Information Technology Specialist,"Đà Nang, Da Nang City, Vietnam",available,2025-03-27 16:15,None,About the job\nCompany Description\n LTD Mater...
3,Taiko Daihung Pte Ltd,https://www.linkedin.com/company/taiko-daihung...,Sales Engineer,"Ho Chi Minh City, Vietnam",available,2025-03-27 16:15,https://www.linkedin.com/in/reanne-gan-a1b498b3,About the job\nWork in a stable company - Inor...
4,Avery Dennison,https://www.linkedin.com/company/avery-denniso...,Social Compliance Engineer,"Nhà Bè, Ho Chi Minh City, Vietnam",available,2025-03-27 16:16,None,About the job\nYOUR RESPONSIBILITIES WILL INCL...


# Vietnamwork

In [26]:
# Đường dẫn tới file JSON đã hợp nhất
vnw_json_path = r'C:\working\job_rcm\data\vietnamwork\detailview\data.json'

# Đọc dữ liệu từ file JSON
with open(vnw_json_path, 'r', encoding='utf-8') as f:
    data = json.load(f)

# Chuyển đổi thành DataFrame
vnw_df = pd.DataFrame(data)
vnw_df = vnw_df.drop(vnw_df[vnw_df['job_status'] == 'error_link'].index)
vnw_df = vnw_df.reset_index(drop=True)
vnw_df.head()

,job_link,company_name,company_href,job_name,salary,region,job_status,scrape_date,hr_id,jd
0,https://www.vietnamworks.com/business-analyst-...,United Insurance Company Of Vietnam,https://www.vietnamworks.com/nha-tuyen-dung/un...,Business Analyst,Thương lượng,Hà Nội,Hết hạn trong 2 ngày,2024-11-12T22:50:34.806117,None,Mô tả công việc\n• Receive and manage all busi...
1,https://www.vietnamworks.com/chuyen-vien-quan-...,Trung Tâm Công Nghệ Thông Tin MobiFone,https://www.vietnamworks.com/nha-tuyen-dung/tr...,Chuyên Viên Quản Lý Sản Phẩm - Ngành CNTT,280tr-400tr ₫/năm,Hà Nội,Hết hạn trong 3 ngày,2024-11-12T22:50:42.098082,None,Mô tả công việc\n• Chủ trì xây dựng chiến lược...
2,https://www.vietnamworks.com/ho-head-of-advanc...,Ngân Hàng TMCP Á Châu (ACB),https://www.vietnamworks.com/nha-tuyen-dung/ng...,HO - Head of Advanced Analytics,Thương lượng,Hồ Chí Minh,Hết hạn trong 1 ngày,2024-11-12T22:51:42.826042,None,"Mô tả công việc\nDesign, implement and deliver..."
3,https://www.vietnamworks.com/system-operations...,Tenzing Pacific Services Ltd.,https://www.vietnamworks.com/nha-tuyen-dung/te...,System Operations Coordinator,8tr-12tr ₫/tháng,Hồ Chí Minh,Hết hạn trong 9 ngày,2024-11-12T22:51:57.940523,None,Mô tả công việc\nOverview:\nWe are looking for...
4,https://www.vietnamworks.com/analytics-and-ins...,Techcombank,https://www.vietnamworks.com/nha-tuyen-dung/te...,Analytics and Insights Analyst,"$ 1,000-2,500 /tháng","Hồ Chí Minh, Hà Nội",Hết hạn trong 8 ngày,2024-11-12T22:52:13.260541,None,Mô tả công việc\nWithin the Analytics and Insi...


In [27]:
vnw_df['scrape_date'] = pd.to_datetime(vnw_df['scrape_date'], format="%Y-%m-%dT%H:%M:%S.%f")
vnw_df['scrape_date'] = vnw_df['scrape_date'].dt.strftime("%Y-%m-%d %H:%M")
vnw_df.head()

,job_link,company_name,company_href,job_name,salary,region,job_status,scrape_date,hr_id,jd
0,https://www.vietnamworks.com/business-analyst-...,United Insurance Company Of Vietnam,https://www.vietnamworks.com/nha-tuyen-dung/un...,Business Analyst,Thương lượng,Hà Nội,Hết hạn trong 2 ngày,2024-11-12 22:50,None,Mô tả công việc\n• Receive and manage all busi...
1,https://www.vietnamworks.com/chuyen-vien-quan-...,Trung Tâm Công Nghệ Thông Tin MobiFone,https://www.vietnamworks.com/nha-tuyen-dung/tr...,Chuyên Viên Quản Lý Sản Phẩm - Ngành CNTT,280tr-400tr ₫/năm,Hà Nội,Hết hạn trong 3 ngày,2024-11-12 22:50,None,Mô tả công việc\n• Chủ trì xây dựng chiến lược...
2,https://www.vietnamworks.com/ho-head-of-advanc...,Ngân Hàng TMCP Á Châu (ACB),https://www.vietnamworks.com/nha-tuyen-dung/ng...,HO - Head of Advanced Analytics,Thương lượng,Hồ Chí Minh,Hết hạn trong 1 ngày,2024-11-12 22:51,None,"Mô tả công việc\nDesign, implement and deliver..."
3,https://www.vietnamworks.com/system-operations...,Tenzing Pacific Services Ltd.,https://www.vietnamworks.com/nha-tuyen-dung/te...,System Operations Coordinator,8tr-12tr ₫/tháng,Hồ Chí Minh,Hết hạn trong 9 ngày,2024-11-12 22:51,None,Mô tả công việc\nOverview:\nWe are looking for...
4,https://www.vietnamworks.com/analytics-and-ins...,Techcombank,https://www.vietnamworks.com/nha-tuyen-dung/te...,Analytics and Insights Analyst,"$ 1,000-2,500 /tháng","Hồ Chí Minh, Hà Nội",Hết hạn trong 8 ngày,2024-11-12 22:52,None,Mô tả công việc\nWithin the Analytics and Insi...


# tích hợp dữ liệu

In [1]:
data_structure = {
    "source": "facebook/linkedin/vietnamwork",  # Nguồn dữ liệu
    "job_id": "Unique ID (dựa trên hr_id hoặc khác)",  # ID bài đăng
    "job_link": "URL to the job post",          # Link bài đăng (nếu có)
    "company_name": "Company Name",            # Tên công ty
    "title": "Job Title (or content for Facebook)", # Tiêu đề công việc
    "region": "Location or group_id",          # Địa điểm
    "status": "Status of job post",            # Trạng thái bài đăng
    "scrape_date": "Date when the data was scraped", # Ngày lấy dữ liệu
    "description": "Job description or content", # Nội dung chi tiết
    "source_specific": {                       # Các trường đặc thù
        "facebook": {
            "author_name": "Author of the post",      # Tác giả bài đăng
            "images": "List of image URLs",          # Hình ảnh đính kèm
            "group_id": "Facebook group ID",         # ID của nhóm Facebook
            "post_date": "Date of the post"          # Ngày đăng bài
        },
        "linkedin": {
            "company_href": "Link to company profile", # Link công ty
            "jd": "Job description (LinkedIn)"         # JD chi tiết
        },
        "vietnamwork": {
            "company_href": "Link to company profile", # Link công ty
            "salary": "Salary information",            # Mức lương
            "jd": "Job description (VietnamWorks)"     # JD chi tiết
        }
    }
}


## chuẩn hóa dữ liệu 

In [34]:
facebook_df = df

In [35]:
facebook_df['source'] = 'facebook'
facebook_df = facebook_df.rename(columns={
    'post_href': 'job_link',
    'author_name': 'source_specific.facebook.author_name',
    'group_id': 'source_specific.facebook.group_id',
    'post_date': 'source_specific.facebook.post_date',
    'content': 'description',
    'images': 'source_specific.facebook.images',
    'scrape_date': 'scrape_date'
})
facebook_df['title'] = None
facebook_df['status'] = None  # Facebook job không có trạng thái
facebook_df['region'] = None


In [36]:
ld_df['source'] = 'linkedin'
ld_df = ld_df.rename(columns={
    'job_link': 'job_link',
    'company_name': 'company_name',
    'company_href': 'source_specific.linkedin.company_href',
    'job_name': 'title',
    'region': 'region',
    'job_status': 'status',
    'jd': 'source_specific.linkedin.jd',
    'scrape_date': 'scrape_date',
    'hr_id': 'job_id'
})
ld_df['description'] = ld_df['source_specific.linkedin.jd']


In [37]:
vnw_df['source'] = 'vietnamwork'
vnw_df = vnw_df.rename(columns={
    'job_link': 'job_link',
    'company_name': 'company_name',
    'company_href': 'source_specific.vietnamwork.company_href',
    'job_name': 'title',
    'salary': 'source_specific.vietnamwork.salary',
    'region': 'region',
    'job_status': 'status',
    'jd': 'source_specific.vietnamwork.jd',
    'scrape_date': 'scrape_date',
    'hr_id': 'job_id'
})
vnw_df['description'] = vnw_df['source_specific.vietnamwork.jd']


In [38]:
print("Facebook Columns:", facebook_df.columns.tolist())
print("LinkedIn Columns:", ld_df.columns.tolist())
print("VietnamWorks Columns:", vnw_df.columns.tolist())

Facebook Columns: ['job_link', 'source_specific.facebook.author_name', 'hr_id', 'source_specific.facebook.group_id', 'scrape_date', 'source_specific.facebook.post_date', 'description', 'source_specific.facebook.images', 'source', 'title', 'status', 'region']
LinkedIn Columns: ['job_link', 'company_name', 'source_specific.linkedin.company_href', 'title', 'region', 'status', 'scrape_date', 'job_id', 'source_specific.linkedin.jd', 'source', 'description']
VietnamWorks Columns: ['job_link', 'company_name', 'source_specific.vietnamwork.company_href', 'title', 'source_specific.vietnamwork.salary', 'region', 'status', 'scrape_date', 'job_id', 'source_specific.vietnamwork.jd', 'source', 'description']


In [39]:
columns = [
    'source', 'job_id', 'job_link', 'company_name', 'title', 'region', 
    'status', 'scrape_date', 'description', 'source_specific'
]


In [40]:
# Xử lý Facebook
missing_columns = set(columns) - set(facebook_df.columns)
for col in missing_columns:
    facebook_df[col] = None

# Xử lý LinkedIn
missing_columns = set(columns) - set(ld_df.columns)
for col in missing_columns:
    ld_df[col] = None

# Xử lý VietnamWorks
missing_columns = set(columns) - set(vnw_df.columns)
for col in missing_columns:
    vnw_df[col] = None


In [41]:
# Đảm bảo các cột tuân theo thứ tự chuẩn hóa
facebook_df = facebook_df[columns]
ld_df = ld_df[columns]
vnw_df = vnw_df[columns]

# Hợp nhất các DataFrame
merged_df = pd.concat([facebook_df, ld_df, vnw_df], ignore_index=True)


In [42]:
merged_df 

,source,job_id,job_link,company_name,title,region,status,scrape_date,description,source_specific
0,facebook,None,https://www.facebook.com/groups/16032510932029...,None,None,None,None,2024-10-24 20:10:29,Inbox mình gửi Jd,None
1,facebook,None,https://www.facebook.com/groups/16032510932029...,None,None,None,None,2024-10-24 20:10:51,TUYỂN DỤNG NHÂN VIÊN XỬ LÝ DỮ LIỆU HÌNH ẢNH - ...,None
2,facebook,None,https://www.facebook.com/groups/16032510932029...,None,None,None,None,2024-10-24 20:11:06,[DAYONE ASIA (SUNNY DAYS PIANO)] - TUYỂN DỤNG\...,None
3,facebook,None,https://www.facebook.com/groups/16032510932029...,None,None,None,None,2024-10-24 20:11:21,"Đừng săn Hồng Hài Nhi nữaaa, săn job XGame đi ...",None
4,facebook,None,https://www.facebook.com/groups/16032510932029...,None,None,None,None,2024-10-24 20:16:53,Inbox mình gửi Jd,None
...,...,...,...,...,...,...,...,...,...,...
9765,vietnamwork,None,https://www.vietnamworks.com/chuyen-vien-chuye...,Ngân Hàng Thương Mại Cổ Phần Quân Đội - MB,Chuyên Viên/ Chuyên Viên Cao Cấp Phân Tích Dữ ...,Hà Nội,Hết hạn trong 4 ngày,2024-11-12 22:59,Mô tả công việc\n• Tham gia nghiên cứu các sản...,None
9766,vietnamwork,None,https://www.vietnamworks.com/business-data-ana...,Gamma Interactive Inc.,Business Data Analyst (Based in Manila/ Philip...,"Hồ Chí Minh, Đà Nẵng",Hết hạn trong 4 ngày,2024-11-12 23:00,"Mô tả công việc\n• Collect, clean, and organiz...",None
9767,vietnamwork,None,https://www.vietnamworks.com/business-analysis...,Prudential Vietnam Assurance,Business Analysis Manager,Hồ Chí Minh,Hết hạn trong 4 ngày,2024-11-12 23:00,Mô tả công việc\nJob Purpose:\n- This role req...,None
9768,vietnamwork,None,https://www.vietnamworks.com/chuyen-vien-chuye...,Ngân Hàng Thương Mại Cổ Phần Quân Đội - MB,Chuyên Viên/ Chuyên Viên Cao Cấp Quản Trị CSDL...,Hà Nội,Hết hạn trong 3 ngày,2024-11-12 23:00,"Mô tả công việc\n• Cấu hình, quản trị, giám sá...",None


# tích hợp thành file .py

dữ liệu mới sẽ được ghi đè lên dữ liệu cũ

#### facebook_preprocessed

In [6]:
import pandas as pd
import os
import json
import re

# Đường dẫn tới thư mục gốc và file kết quả
base_dir = r"C:\working\job_rcm\data\facebook"
output_path = r'C:\working\job_rcm\data\preprocessed\facebook\preprocessed_data.csv'
merged_output_path = r'C:\working\job_rcm\data\preprocessed\facebook\merged_data.json'
file_list_path = r'C:\working\job_rcm\data\preprocessed\facebook\file_list.txt'

# Hàm lấy danh sách tất cả các file JSON
def get_all_json_files(base_dir):
    json_files = []
    for root, dirs, files in os.walk(base_dir):
        for file in files:
            if file == 'data.json':
                json_files.append(os.path.join(root, file))
    return json_files

# Lấy danh sách file JSON hiện tại
current_file_list = get_all_json_files(base_dir)

# Kiểm tra xem file_list.txt có tồn tại không
if os.path.exists(file_list_path):
    # Đọc danh sách file cũ
    with open(file_list_path, 'r') as f:
        previous_file_list = f.read().splitlines()
else:
    # Nếu chưa có file_list.txt, coi như không có file nào trước đó
    previous_file_list = []
    # Tạo file_list.txt với nội dung trống
    with open(file_list_path, 'w') as f:
        f.write("")

# So sánh danh sách file để kiểm tra tệp mới
if set(current_file_list) != set(previous_file_list):
    # Có file mới, thực hiện gộp file (Task 1)
    all_posts = []
    for file_path in current_file_list:
        with open(file_path, 'r', encoding='utf-8') as f:
            data = json.load(f)
            all_posts.append(data)
    
    # Ghi dữ liệu hợp nhất ra file
    with open(merged_output_path, 'w', encoding='utf-8') as f:
        json.dump(all_posts, f, ensure_ascii=False, indent=4)
    
    # Cập nhật file_list.txt với danh sách file hiện tại
    with open(file_list_path, 'w') as f:
        f.write("\n".join(current_file_list))
    
    print(f"Dữ liệu mới đã được hợp nhất và lưu vào '{merged_output_path}'")
else:
    print("Không có file mới, đọc trực tiếp từ tệp JSON đã hợp nhất.")

# Đọc dữ liệu từ file JSON hợp nhất
facebook_df = pd.read_json(merged_output_path)
print("Dữ liệu đã được đọc thành công.")

facebook_df['post_date'] = facebook_df['post_date'].apply(
    lambda x: re.sub(r"Thứ \w+, (\d+) Tháng (\d+), (\d+) lúc (\d+:\d+)", r"\3-\2-\1 \4", x) if pd.notnull(x) else x
)
facebook_df['post_date'] = pd.to_datetime(facebook_df['post_date'], format='%Y-%m-%d %H:%M', errors='coerce')

def fill_nat_with_timedelta(group):
    last_valid = None
    result = []
    for date in group:
        if pd.isna(date):
            last_valid -= pd.Timedelta(minutes=30)
            result.append(last_valid)
        else:
            last_valid = date
            result.append(last_valid)
    return result

facebook_df['post_date'] = facebook_df.groupby('group_id')['post_date'].transform(fill_nat_with_timedelta)

facebook_df.to_csv(output_path, index=False, encoding='utf-8')
print(f"Dữ liệu đã được lưu vào: {output_path}")



Không có file mới, đọc trực tiếp từ tệp JSON đã hợp nhất.
Dữ liệu đã được đọc thành công.
Dữ liệu đã được lưu vào: C:\working\job_rcm\data\preprocessed\facebook\preprocessed_data.csv


#### linkedin_preprocessed

In [41]:
import pandas as pd
import os
import json

# Đường dẫn
base_dir = r"C:\working\job_rcm\data\linkedin"
merged_output_path = r'C:\working\job_rcm\data\preprocessed\linkedin\merged_data.json'
csv_output_path = r'C:\working\job_rcm\data\preprocessed\linkedin\preprocessed_data.csv'

# Bước 1: Duyệt qua các thư mục con chứa "Detailview" và lấy data.json
all_ld_posts = []

for root, dirs, files in os.walk(base_dir):
    if 'Detailview' in root:  # Chỉ lấy các thư mục chứa "Detailview"
        for file in files:
            if file == 'data.json':
                file_path = os.path.join(root, file)
                with open(file_path, 'r', encoding='utf-8') as f:
                    try:
                        data = json.load(f)
                        all_ld_posts.append(data)
                    except Exception as e:
                        print(f"Lỗi khi đọc {file_path}: {e}")

# Bước 2: Ghi dữ liệu đã hợp nhất ra file JSON
with open(merged_output_path, 'w', encoding='utf-8') as f:
    json.dump(all_ld_posts, f, ensure_ascii=False, indent=4)

print(f"Dữ liệu đã được hợp nhất thành công và lưu vào file '{merged_output_path}'")

# Bước 3: Đọc lại dữ liệu từ file JSON
with open(merged_output_path, 'r', encoding='utf-8') as f:
    data = json.load(f)

# Bước 4: Trích xuất danh sách công việc (job)
all_ld_jobs = [job for item in data if "job" in item for job in item["job"]]

# Bước 5: Chuyển sang DataFrame
ld_df = pd.DataFrame(all_ld_jobs)

# Bước 6: Xử lý thời gian scrape_date nếu có
if 'scrape_date' in ld_df.columns:
    ld_df['scrape_date'] = pd.to_datetime(ld_df['scrape_date'], format="%Y-%m-%dT%H:%M:%S.%f", errors='coerce')
    ld_df['scrape_date'] = ld_df['scrape_date'].dt.strftime("%Y-%m-%d %H:%M")

# Bước 7: Lưu ra file CSV
ld_df.to_csv(csv_output_path, index=False, encoding='utf-8')
print(f"Dữ liệu đã được lưu vào: {csv_output_path}")


Dữ liệu đã được hợp nhất thành công và lưu vào file 'C:\working\job_rcm\data\preprocessed\linkedin\merged_data.json'
Dữ liệu đã được lưu vào: C:\working\job_rcm\data\preprocessed\linkedin\preprocessed_data.csv


#### vietnamwork_preprocessed

In [8]:
import pandas as pd
import os
import json

# Đường dẫn tới file JSON
vnw_json_path = r'C:\working\job_rcm\data\vietnamwork\detailview\data.json'
output_path = r'C:\working\job_rcm\data\preprocessed\vietnamwork\preprocessed_data.csv'

# Đọc dữ liệu từ file JSON
with open(vnw_json_path, 'r', encoding='utf-8') as f:
    data = json.load(f)

# Chuyển đổi thành DataFrame
vnw_df = pd.DataFrame(data)
vnw_df = vnw_df.drop(vnw_df[vnw_df['job_status'] == 'error_link'].index)
vnw_df = vnw_df.reset_index(drop=True)

vnw_df['scrape_date'] = pd.to_datetime(vnw_df['scrape_date'], format="%Y-%m-%dT%H:%M:%S.%f")
vnw_df['scrape_date'] = vnw_df['scrape_date'].dt.strftime("%Y-%m-%d %H:%M")


vnw_df.to_csv(output_path, index=False, encoding='utf-8')
print(f"Dữ liệu đã được lưu vào: {output_path}")


Dữ liệu đã được lưu vào: C:\working\job_rcm\data\preprocessed\vietnamwork\preprocessed_data.csv


#### intergration.py

In [13]:
import pandas as pd
import os

# Đường dẫn
output_path = r'C:\working\job_rcm\data\preprocessed\final.csv'

fb_path = r'C:\working\job_rcm\data\preprocessed\facebook\preprocessed_data.csv'
ld_path = r'C:\working\job_rcm\data\preprocessed\linkedin\preprocessed_data.csv'
vnw_path = r'C:\working\job_rcm\data\preprocessed\vietnamwork\preprocessed_data.csv'

# Đọc dữ liệu
facebook_df = pd.read_csv(fb_path)
ld_df = pd.read_csv(ld_path)
vnw_df = pd.read_csv(vnw_path)

# Chuẩn hóa các DataFrame
# Facebook
facebook_df['source'] = 'facebook'
facebook_df = facebook_df.rename(columns={
    'post_href': 'job_link',
    'content': 'description'
})
facebook_df['hr_id'] = facebook_df.get('hr_id', None)
facebook_df['post_date'] = facebook_df.get('post_date', None)
facebook_df['company_name'] = None
facebook_df['company_href'] = None
facebook_df['job_name'] = None
facebook_df['salary'] = None
facebook_df['region'] = None
facebook_df['job_status'] = None
facebook_df['jd'] = None

# LinkedIn
ld_df['source'] = 'linkedin'
ld_df = ld_df.rename(columns={
    'job_link': 'job_link',
    'company_name': 'company_name',
    'company_href': 'company_href',
    'job_name': 'job_name',
    'region': 'region',
    'job_status': 'job_status',
    'scrape_date': 'scrape_date',
    'hr_id': 'hr_id',
    'jd': 'jd'
})
ld_df['description'] = None
ld_df['salary'] = None
ld_df['post_date'] = None

# VietnamWorks
vnw_df['source'] = 'vietnamwork'
vnw_df = vnw_df.rename(columns={
    'job_link': 'job_link',
    'company_name': 'company_name',
    'company_href': 'company_href',
    'job_name': 'job_name',
    'salary': 'salary',
    'region': 'region',
    'job_status': 'job_status',
    'scrape_date': 'scrape_date',
    'hr_id': 'hr_id',
    'jd': 'jd'
})
vnw_df['description'] = None
vnw_df['post_date'] = None

# Danh sách cột chuẩn hóa
columns = [
    'source', 'job_link', 'hr_id', 'company_name', 'company_href', 
    'job_name', 'salary', 'region', 'job_status','post_date', 'scrape_date', 'jd', 'description'
]

# Thêm cột thiếu và sắp xếp thứ tự
facebook_df = facebook_df.reindex(columns=columns)
ld_df = ld_df.reindex(columns=columns)
vnw_df = vnw_df.reindex(columns=columns)

# Gộp các DataFrame
merged_df = pd.concat([facebook_df, ld_df, vnw_df], ignore_index=True)

# Xuất dữ liệu
merged_df.to_csv(output_path, index=False, encoding='utf-8')
print(f"Dữ liệu đã được lưu vào: {output_path}")

Dữ liệu đã được lưu vào: C:\working\job_rcm\data\preprocessed\final.csv


In [1]:
import pandas as pd 
df = pd.read_csv(r'C:\working\job_rcm\data\preprocessed\final.csv')
print(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5807 entries, 0 to 5806
Data columns (total 13 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   source        5807 non-null   object
 1   job_link      5807 non-null   object
 2   hr_id         3098 non-null   object
 3   company_name  3991 non-null   object
 4   company_href  3991 non-null   object
 5   job_name      3991 non-null   object
 6   salary        28 non-null     object
 7   region        3991 non-null   object
 8   job_status    3991 non-null   object
 9   post_date     1816 non-null   object
 10  scrape_date   5807 non-null   object
 11  jd            3991 non-null   object
 12  description   1813 non-null   object
dtypes: object(13)
memory usage: 589.9+ KB
None


In [3]:
df_sample = df.sample(n=1000, random_state=None)
df_sample

,source,job_link,hr_id,company_name,company_href,job_name,salary,region,job_status,post_date,scrape_date,jd,description
49,facebook,https://www.facebook.com/groups/16032510932029...,https://www.facebook.com/groups/160325109320298/,NaN,NaN,NaN,NaN,NaN,NaN,2024-10-03 09:59:00,2024-10-24 20:28:54,NaN,Tạo web con MIỄN PHÍ bán proxy tự động.\n+ Khô...
3228,linkedin,https://www.linkedin.com/jobs/view/3695849255/...,NaN,Per Scholas,https://www.linkedin.com/company/perscholas/life,Software Engineering Instructor (Continuous),NaN,Hanoi Capital Region,available,NaN,2024-11-08 15:32,About the job\nABOUT PER SCHOLAS:\n\nFor nearl...,NaN
1837,linkedin,https://www.linkedin.com/jobs/view/3988597221/...,NaN,NielsenIQ,https://www.linkedin.com/company/nielseniq/life,Data Processing Specialist,NaN,"Ho Chi Minh City, Vietnam",available,NaN,2024-11-08 23:51,About the job\nJob Description\n\nThe Referenc...,NaN
1769,facebook,https://www.facebook.com/groups/16032510932029...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2022-10-28 17:44:00,2024-10-26 08:05:39,NaN,Mình có khoá DA và DE của Cole. Học qua drive ...
2118,linkedin,https://www.linkedin.com/jobs/view/4068646478/...,NaN,AhaSlides - Audience Engagement Platform,https://www.linkedin.com/company/ahaslides/life,Senior Backend Developer,NaN,"Hanoi, Hanoi, Vietnam",available,NaN,2024-11-09 00:28,"About the job\nWe are AhaSlides, a SaaS (softw...",NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
4240,linkedin,https://www.linkedin.com/jobs/view/4031329408/...,NaN,Ecolab,https://www.linkedin.com/company/ecolab/life,Food Safety & Brand Standard Auditor (Freelancer),NaN,Ho Chi Minh City Metropolitan Area,available,NaN,2024-11-08 19:40,About the job\nEcosure - a division of Ecolab ...,NaN
1588,facebook,https://www.facebook.com/groups/16032510932029...,https://www.facebook.com/profile.php?id=100077...,NaN,NaN,NaN,NaN,NaN,NaN,2022-11-01 12:14:00,2024-10-26 04:31:35,NaN,"HN - Tìm gấp #Data Engineer 2y kn, join dự án ..."
86,facebook,https://www.facebook.com/groups/16032510932029...,https://www.facebook.com/groups/160325109320298/,NaN,NaN,NaN,NaN,NaN,NaN,2024-09-20 08:31:00,2024-10-24 20:42:00,NaN,[HN] Tuyển dụng Data Engineer – Junior\nYC:\n-...
786,facebook,https://www.facebook.com/groups/16032510932029...,https://www.facebook.com/groups/160325109320298/,NaN,NaN,NaN,NaN,NaN,NaN,2024-08-15 21:38:00,2024-10-25 09:34:06,NaN,ib mình vs ạ


In [4]:
df_sample.to_csv(r'C:\working\job_rcm\data\preprocessed\final_sample.csv', index=False, encoding='utf-8')

In [15]:
df['job_name']

0                                                     NaN
1                                                     NaN
2                                                     NaN
3                                                     NaN
4                                                     NaN
                              ...                        
5802    Chuyên Viên/ Chuyên Viên Cao Cấp Phân Tích Dữ ...
5803    Business Data Analyst (Based in Manila/ Philip...
5804                            Business Analysis Manager
5805    Chuyên Viên/ Chuyên Viên Cao Cấp Quản Trị CSDL...
5806                      Sales Data Analyst (Healthcare)
Name: job_name, Length: 5807, dtype: object

# Update preprocess

In [ ]:
import pandas as pd
import os
import json
import re
import gspread
from google.oauth2 import service_account
from datetime import datetime
import uuid

# Google Sheet setup
SCOPES = ["https://www.googleapis.com/auth/spreadsheets", "https://www.googleapis.com/auth/drive"]
SERVICE_ACCOUNT_FILE = r'C:\working\job_rcm\job_rcm_code\job_scraping\job-rcm-luan-0e530aa9b6a0.json'

credentials = service_account.Credentials.from_service_account_file(
    SERVICE_ACCOUNT_FILE, scopes=SCOPES)
client = gspread.authorize(credentials)
sheet = client.open_by_key("1cXvQDtzkcjoYaZ0F2tVqwpLaS14M_00m2Qi8zDAmy_o").worksheet("logs")


####################################
# Log system code ##################
####################################

# Function to generate a unique log ID
def generate_log_id(scraping_type):
    timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
    random_suffix = str(uuid.uuid4())[:3]
    return f"FBPR_{timestamp}_{random_suffix}_{scraping_type}"

# Function to write a start log entry
def write_start_log(log_id, scraping_type, keywords=""):
    sheet.append_row([
            log_id,
            'Facebook',
            scraping_type,
            'Started',
            datetime.now().strftime('%Y-%m-%d %H:%M:%S'),  # Start time
            '',  # End time (empty for start log)
            0,   # Duration (0 for start log)
            0,   # Total keywords (will be updated in end log)
            0,   # Total jobs (will be updated in end log)
            0,   # New jobs (will be updated in end log)
            0,   # Error count (will be updated in end log)
            '',  # Error details (will be updated in end log)
            keywords  # Keywords
        ])
    return log_id

# Function to write an end log entry
def write_end_log(log_id, scraping_type, start_time, status, total_keywords=0, total_jobs=0,new_jobs=0, error_count=0, error_details="", keywords=""):
    end_time = datetime.now()
    start_datetime = datetime.strptime(start_time, '%Y-%m-%d %H:%M:%S')
    duration = int((end_time - start_datetime).total_seconds())
    
    sheet.append_row([
            log_id,
            'Facebook',
            scraping_type,
            status,
            start_time,
            end_time.strftime('%Y-%m-%d %H:%M:%S'),
            duration,
            total_keywords,
            total_jobs,
            new_jobs,
            error_count,
            error_details,
            keywords
        ])


preprocess_log_id = generate_log_id("preprocess")
preprocess_start_time = datetime.now().strftime('%Y-%m-%d %H:%M:%S')
scraping_type = "preprocess"
# Ghi log bắt đầu
write_start_log(preprocess_log_id, scraping_type, keywords="Facebook Preprocessing")



In [5]:

# Đường dẫn tới thư mục gốc và file kết quả
base_dir = r"C:\working\job_rcm\data\facebook"
output_path = r'C:\working\job_rcm\data\preprocessed\facebook\preprocessed_data.csv'
merged_output_path = r'C:\working\job_rcm\data\preprocessed\facebook\merged_data.json'
file_list_path = r'C:\working\job_rcm\data\preprocessed\facebook\file_list.txt'

# Hàm lấy danh sách tất cả các file JSON
def get_all_json_files(base_dir):
    json_files = []
    for root, dirs, files in os.walk(base_dir):
        for file in files:
            if file == 'data.json':
                json_files.append(os.path.join(root, file))
    return json_files

# Lấy danh sách file JSON hiện tại
current_file_list = get_all_json_files(base_dir)

# Kiểm tra xem file_list.txt có tồn tại không
if os.path.exists(file_list_path):
    # Đọc danh sách file cũ
    with open(file_list_path, 'r') as f:
        previous_file_list = f.read().splitlines()
else:
    # Nếu chưa có file_list.txt, coi như không có file nào trước đó
    previous_file_list = []
    # Tạo file_list.txt với nội dung trống
    with open(file_list_path, 'w') as f:
        f.write("")

# So sánh danh sách file để kiểm tra tệp mới
if set(current_file_list) != set(previous_file_list):
    # Có file mới, thực hiện gộp file (Task 1)
    all_posts = []
    for file_path in current_file_list:
        with open(file_path, 'r', encoding='utf-8') as f:
            data = json.load(f)
            all_posts.append(data)
    
    # Ghi dữ liệu hợp nhất ra file
    with open(merged_output_path, 'w', encoding='utf-8') as f:
        json.dump(all_posts, f, ensure_ascii=False, indent=4)
    
    # Cập nhật file_list.txt với danh sách file hiện tại
    with open(file_list_path, 'w') as f:
        f.write("\n".join(current_file_list))
    
    print(f"Dữ liệu mới đã được hợp nhất và lưu vào '{merged_output_path}'")
    msg = f"Dữ liệu mới đã được hợp nhất và lưu vào '{merged_output_path}'"
else:
    print("Không có file mới, đọc trực tiếp từ tệp JSON đã hợp nhất.")
    msg = "Không có file mới, đọc trực tiếp từ tệp JSON đã hợp nhất."

# Đọc dữ liệu từ file JSON hợp nhất
facebook_df = pd.read_json(merged_output_path)
print("Dữ liệu đã được đọc thành công.")
# print(facebook_df)

facebook_df['post_date'] = facebook_df['post_date'].apply(
    lambda x: re.sub(r"Thứ \w+, (\d+) Tháng (\d+), (\d+) lúc (\d+:\d+)", r"\3-\2-\1 \4", x) if pd.notnull(x) else x
)
facebook_df['post_date'] = pd.to_datetime(facebook_df['post_date'], format='%Y-%m-%d %H:%M', errors='coerce')

def fill_nat_with_timedelta(group):
    last_valid = None
    result = []
    for date in group:
        if pd.isna(date):
            last_valid -= pd.Timedelta(minutes=30)
            result.append(last_valid)
        else:
            last_valid = date
            result.append(last_valid)
    return result

# facebook_df['post_date'] = facebook_df.groupby('group_id')['post_date'].transform(fill_nat_with_timedelta)

facebook_df.to_csv(output_path, index=False, encoding='utf-8')
print(f"Dữ liệu đã được lưu vào: {output_path}")

write_end_log(
    preprocess_log_id,
    scraping_type,
    preprocess_start_time,
    'Completed',
    total_keywords=0,
    total_jobs=0,
    new_jobs=0,
    error_count=0,
    error_details="",
    keywords=msg
)

Dữ liệu mới đã được hợp nhất và lưu vào 'C:\working\job_rcm\data\preprocessed\facebook\merged_data.json'
Dữ liệu đã được đọc thành công.
Dữ liệu đã được lưu vào: C:\working\job_rcm\data\preprocessed\facebook\preprocessed_data.csv


In [2]:
facebook_df

""
